<a href="https://colab.research.google.com/github/Ishika-Pattnaik/Reviews-sentiment-analysis/blob/main/distilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install transformers torch pandas numpy matplotlib seaborn scikit-learn accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 74.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: matplotlib
    Found existin

In [9]:
import tarfile

dataset_path = '/content/sample_data/aclImdb_v1.tar'
extract_path = '/content/'

with tarfile.open(dataset_path, 'r') as tar:
    tar.extractall(path=extract_path)

print("✅ Dataset extracted successfully!")

/tmp/ipython-input-2263771871.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_path)


✅ Dataset extracted successfully!


In [10]:
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizer
import os

# Define custom dataset class
class IMDbDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load IMDb data
def load_imdb_data(data_dir):
    texts, labels = [], []
    for label in ['pos', 'neg']:
        label_dir = os.path.join(data_dir, label)
        for file in os.listdir(label_dir):
            with open(os.path.join(label_dir, file), 'r', encoding='utf-8') as f:
                texts.append(f.read())
                labels.append(1 if label == 'pos' else 0)
    return texts, labels

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load train and test data
train_dir = '/content/aclImdb/train'
test_dir = '/content/aclImdb/test'
train_texts, train_labels = load_imdb_data(train_dir)
test_texts, test_labels = load_imdb_data(test_dir)

# Create datasets
train_dataset = IMDbDataset(train_texts, train_labels, tokenizer, max_len=128)
test_dataset = IMDbDataset(test_texts, test_labels, tokenizer, max_len=128)

# Save datasets
torch.save(train_dataset, '/content/train_dataset.pt')
torch.save(test_dataset, '/content/test_dataset.pt')
print("✅ Datasets tokenized and saved!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

✅ Datasets tokenized and saved!


In [ ]:
import os
import torch
import numpy as np
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset

# Disable W&B logging (set before any imports that might trigger it)
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

# Define IMDbDataset class (needed for loading saved datasets)
class IMDbDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Pre-download model and tokenizer to avoid API prompts
model_name = 'distilbert-base-uncased'
local_model_dir = '/content/distilbert_local'
if not os.path.exists(local_model_dir):
    print("Downloading model and tokenizer locally...")
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
    tokenizer.save_pretrained(local_model_dir)
    model.save_pretrained(local_model_dir)
    print(f"✅ Model and tokenizer saved to {local_model_dir}")
else:
    print(f"✅ Using cached model and tokenizer from {local_model_dir}")

# Load model and tokenizer from local directory
model = DistilBertForSequenceClassification.from_pretrained(local_model_dir, num_labels=2)
tokenizer = DistilBertTokenizer.from_pretrained(local_model_dir)

# Load datasets with weights_only=False
train_dataset = torch.load('/content/train_dataset.pt', weights_only=False)
test_dataset = torch.load('/content/test_dataset.pt', weights_only=False)

# Optimized training arguments (with report_to="none" to disable W&B)
training_args = TrainingArguments(
    output_dir='/content/distilbert_imdb_results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    fp16=True,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"  # Disable all external loggers like W&B
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda pred: {
        'accuracy': accuracy_score(pred.label_ids, np.argmax(pred.predictions, axis=1)),
        'f1': precision_recall_fscore_support(pred.label_ids, np.argmax(pred.predictions, axis=1), average='weighted')[2]
    }
)

# Train model
trainer.train()
model.save_pretrained('/content/distilbert_imdb_results')
print("✅ Model trained and saved!")

✅ Using cached model and tokenizer from /content/distilbert_local


In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import Trainer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Load dataset and model
test_dataset = torch.load('/content/test_dataset.pt')
trainer = Trainer(model=DistilBertForSequenceClassification.from_pretrained('/content/distilbert_imdb_results'))

# Evaluate
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])
precision_avg, recall_avg, f1_avg, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
cm = confusion_matrix(y_true, y_pred)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Weighted F1: {f1_avg:.4f}")
print(f"Class-wise Metrics:")
for i, class_name in enumerate(['Negative', 'Positive']):
    print(f"{class_name}: Precision={precision[i]:.4f}, Recall={recall[i]:.4f}, F1={f1[i]:.4f}")

# Save results
results_df = pd.DataFrame({
    'metric': ['accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted'],
    'value': [accuracy, f1_avg, precision_avg, recall_avg]
})
results_df.to_csv('/content/distilbert_results.csv')
cm_df = pd.DataFrame(cm, columns=['Predicted_Negative', 'Predicted_Positive'], index=['Actual_Negative', 'Actual_Positive'])
cm_df.to_csv('/content/distilbert_confusion_matrix.csv')
print("✅ Evaluation complete. Results saved to /content/distilbert_results.csv")